In [ ]:
!pip install streamlit
!pip install pyngrok
!pip install uvicorn
!pip install fastapi


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 101.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.9 MB/s eta 0:00:00


In [ ]:
%%writefile main.py
import streamlit as st
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import load_model
from PIL import Image
import numpy as np
import os
import gdown

# Функция для обучения модели
def train_model(epochs):
    (train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
    train_images, test_images = train_images / 255.0, test_images / 255.0

    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))

    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10))

    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    model.fit(train_images, train_labels, epochs=epochs,
              validation_data=(test_images, test_labels))
    model.save('cifar10_model.h5')
    return model

# Названия классов CIFAR-10
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

# Интерфейс Streamlit
st.title('CIFAR-10 Image Classification')
st.write('Upload an image to classify it using the trained model.')

# Выбор количества эпох
epochs = st.slider('Select number of epochs for training:', 1, 50, 10)

# Обучение модели
if st.button('Train Model'):
    with st.spinner(f'Training the model for {epochs} epochs...'):
        model = train_model(epochs)
        st.success('Model trained and saved!')

# Функция для загрузки модели с Google Диска
def download_model_from_gdrive(file_id, destination):
    url = f'https://drive.google.com/uc?id={file_id}'
    gdown.download(url, destination, quiet=False)

# Проверка наличия сохраненной модели
model = None
model_path = 'cifar10_model.h5'
gdrive_file_id = '1z9Wjw30MfWaiFksPVU9PR3LZcMClxmkD'

if not os.path.exists(model_path):
    with st.spinner('Downloading pre-trained model...'):
        download_model_from_gdrive(gdrive_file_id, model_path)
        st.success('Model downloaded!')

# Загрузка предобученной модели
if os.path.exists(model_path):
    model = load_model(model_path)

if st.button('Load Pre-trained Model'):
    if os.path.exists(model_path):
        model = load_model(model_path)
        st.success('Pre-trained model loaded!')
    else:
        st.error('Pre-trained model not found.')

# Функция для предсказания класса изображения
def predict_image(image):
    image = image.resize((32, 32))
    image = np.array(image) / 255.0
    image = np.expand_dims(image, axis=0)
    predictions = model.predict(image)
    predicted_class = class_names[np.argmax(predictions)]
    return predicted_class

# Загрузка изображения пользователем
uploaded_file = st.file_uploader('Choose an image...', type=['jpg', 'jpeg', 'png'])

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption='Uploaded Image', use_column_width=True)
    st.write("")
    st.write("Classifying...")
    if model:
        predicted_class = predict_image(image)
        st.write(f'The model predicts this image is a {predicted_class}.')
    else:
        st.error('Model is not loaded. Please train the model or load the pre-trained model first.')


Writing main.py


In [ ]:
# Запуск Streamlit
!nohup streamlit run main.py --server.port 8000 &

# Настройка проброса порта
!ssh -o "StrictHostKeyChecking no" -R 80:localhost:8000 nokey@localhost.run


nohup: appending output to 'nohup.out'

Welcome to localhost.run!

Follow your favourite reverse tunnel at [https://twitter.com/localhost_run].

To set up and manage custom domains go to https://admin.localhost.run/

More details on custom domains (and how to enable subdomains of your custom
domain) at https://localhost.run/docs/custom-domains

If you get a permission denied error check the faq for how to connect with a key or
create a free tunnel without a key at [http://localhost:3000/docs/faq#generating-an-ssh-key].

To explore using localhost.run visit the documentation site:
https://localhost.run/docs/


** your connection id is 7f8a5d75-1fd3-4483-90d5-2fbdd72ac8fa, please mention it if you send me a message about an issue. **

authenticated as anonymous user
d5875b30a85f15.lhr.life tunneled with tls termination, https://d5875b30a85f15.lhr.life
create an account and add your key for a longer lasting domain name. see https://localhost.run/docs/forever-free/ for more information.
Op